In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from scipy.stats import skew, kurtosis

def color_moments(image):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Split the channels
    h, s, v = cv2.split(hsv_image)
    
    # Calculate the moments for each channel
    h_mean = np.mean(h)
    s_mean = np.mean(s)
    v_mean = np.mean(v)
    
    h_std = np.std(h)
    s_std = np.std(s)
    v_std = np.std(v)
    
    h_skewness = skew(h.flatten())
    s_skewness = skew(s.flatten())
    v_skewness = skew(v.flatten())
    
    h_kurtosis = kurtosis(h.flatten())
    s_kurtosis = kurtosis(s.flatten())
    v_kurtosis = kurtosis(v.flatten())
    
    # Concatenate the moments into a feature vector
    feature_vector = [h_mean, s_mean, v_mean, h_std, s_std, v_std, h_skewness, s_skewness, v_skewness, h_kurtosis, s_kurtosis, v_kurtosis]
    
    return feature_vector

def process_images(folder_path):
    feature_vectors = []
    labels = []
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            image = cv2.imread(file_path)
            if image is not None:
                feature_vector = color_moments(image)
                feature_vectors.append(feature_vector)
                if "healthy" in subdir:
                    labels.append("healthy")
                elif "infected" in subdir:
                    labels.append("infected")
    return feature_vectors, labels

# Path to the dataset folders
dataset_path = "datasets-B"

# Process healthy datasets
healthy_folder = os.path.join(dataset_path, "healthy")
healthy_features, healthy_labels = process_images(healthy_folder)

# Process infected datasets
infected_folder = os.path.join(dataset_path, "infected")
infected_features, infected_labels = process_images(infected_folder)

# Combine features and labels
all_features = healthy_features + infected_features
all_labels = healthy_labels + infected_labels

# Create a DataFrame
df = pd.DataFrame(all_features, columns=["Hue_Mean", "Saturation_Mean", "Value_Mean",
                                         "Hue_Std", "Saturation_Std", "Value_Std",
                                         "Hue_Skewness", "Saturation_Skewness", "Value_Skewness",
                                         "Hue_Kurtosis", "Saturation_Kurtosis", "Value_Kurtosis"])
df["Label"] = all_labels

# Save to Excel
output_file = "Color-MomentsFE-HSV.xlsx"
df.to_excel(output_file, index=False)
print("Data saved to", output_file)


Data saved to Color-MomentsFE-HSV.xlsx
